Pandas importieren und Excel Datei in DataFrame einlesen

In [11]:
import pandas as pd

df = pd.read_excel("../Lets Meet DB Dump.xlsx")
print(df.head())

    Nachname, Vorname                            Straße Nr, PLZ Ort  \
0     Forster, Martin            Minslebener Str. 0, 46286, Dorsten   
1  Elina, Tsanaklidou               Gartenweg 13, 69126, Heidelberg   
2       Vildan, şahin  Heinrich-Heine-Straße 99c, 17489, Greifswald   
3      Bäumker, Ellen                 Weidenring 39, 81539, München   
4  Bahadır, Bekiroğlu   Eichendorffstraße 20, 79379, Müllheim Baden   

         Telefon  \
0     02372 8020   
1  06221 / 98689   
2  03834 / 22951   
3  0162 / 249788   
4  (07631) 67955   

  Hobby1 %Prio1%; Hobby2 %Prio2%; Hobby3 %Prio3%; Hobby4 %Prio4%; Hobby5 %Prio5%;  \
0  Fremdsprachenkenntnisse erweitern %78%; Im Was...                                
1  Jemanden massieren %57%; Mir die Probleme von ...                                
2  In der Stadt herumbummeln %94%; Mit Freunden z...                                
3  An einer Gruppenreise teilnehmen %97%; Seine f...                                
4  Ins Grüne fahren %23%; H

Saubere Namensvergaben der Spalten

In [12]:
rename_map = {
    "Nachname, Vorname": "name",
    "Straße Nr, PLZ Ort": "adresse",
    "Telefon": "telefon",
    "Hobby1 %Prio1%; Hobby2 %Prio2%; Hobby3 %Prio3%; Hobby4 %Prio4%; Hobby5 %Prio5%;": "hobbys",
    "E-Mail": "email",
    "Geschlecht (m/w/nonbinary)": "geschlecht",
    "Interessiert an": "interessiert_an",
    "Geburtsdatum": "geburtsdatum"
}

df = df.rename(columns=rename_map)


Bereinigung der Zellen

In [ ]:
# 1.1   Führende und folgende Leerzeichen werden mit strip() entfernt. Außerdem werden leere Felder mit NA gefüllt durch pd.NA.
for c in df.columns:
    if df[c].dtype == "object":
        df[c] = df[c].str.strip()
df = df.replace({"": pd.NA})

# 1.2   E-Mail Adressen werden durch .str.lower() einheitlich klein geschrieben und mit str.strip() wird sicher gegangen, dass alle Leerzeichen am Anfang und Ende entfernt sind
#       In der Spalte Telefon werden mit str.replace(r"[^\d+]", "", regex=True) alle Zeichen, außer die Zahlen 0-9 (\d) und das Plus entfernt. r vor der eckigen Klammer sorgt dafür, dass Python die Backslashes ignoriert. regex=True bedeutet, dass der Ausdruck als Regex-Ausdruck behandelt werden soll, was eine Sprache für Muster in Texten ist.
df["email"] = df["email"].str.lower().str.strip()
df["telefon"] = df["telefon"].str.replace(r"[^\d+]", "", regex=True)

# 1.3 Geschlecht normalisieren (m/w → Männlich/Weiblich)
df["geschlecht_norm"] = (
    df["geschlecht"].str.lower()
    .replace({"m":"männlich","w":"weiblich"})
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
    .str.capitalize()
)

# 1.4 Namen/Adresse bei Bedarf trennen (nur falls nötig)
# Name -> Vorname/Nachname (einfaches Pattern)
if "vorname" not in df and "nachname" not in df and "name" in df:
    parts = df["name"].fillna("").str.extract(r"^(?P<vorname>\S+)\s+(?P<nachname>.*)$")
    df["vorname"], df["nachname"] = parts["vorname"], parts["nachname"]

# Adresse -> Straße/Hausnummer/PLZ/Ort (nur falls alles in 'adresse' steckt)
if "adresse" in df and ("straße" not in df or "hausnummer" not in df):
    # sehr vereinfachtes Muster – ggf. an eure Daten anpassen
    df["plz"] = df["adresse"].str.extract(r"\b(\d{5})\b")
    df["ort"] = df["adresse"].str.extract(r"\b\d{5}\s+(.+)$")
    parts = df["adresse"].str.extract(r"^(?P<straße>.+?)\s+(?P<hausnummer>\S+)")
    df["straße"], df["hausnummer"] = parts["straße"], parts["hausnummer"]

# 1.5 Hobbys: Kommaliste in Array
df["hobbies_list"] = df["hobbys"].fillna("").str.split(r"\s*,\s*", regex=True)
